# 腾讯云机器翻译API调用

In [1]:
import json
import pandas as pd
# 导入腾讯云机器翻译API相关模块
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.tmt.v20180321 import tmt_client, models

In [2]:
# 读取列名并以列表的形式返回内容
def read_column(source_file, source_column):
    source_df = pd.read_excel(source_file) # 读取原文件为一个DataFrame
    # 将表头进行翻译
    # source_df.columns=["原文标题", "研究区位置", "时间跨度", "研究方法", "研究结果"]
    words = source_df[source_column]
    words_list = words.tolist() # 转换为列表
    return words_list

In [3]:
# 定义密钥对
SecretId = "AKIDpW7YlusdAUbVAbnkTFn5KUXURGuPjtwo"
SecretKey = "hIivtfAxXyGRXTHvMfswthnvujVBh0Gi"

In [4]:
# 批量翻译
def translate_words(source_file, source_column):
    # 实例化一个认证对象，入参需要传入腾讯云账户 SecretId 和 SecretKey
    cred = credential.Credential(SecretId, SecretKey)
    # 实例化一个http选项，可选的，没有特殊需求可以跳过
    httpProfile = HttpProfile()
    httpProfile.endpoint = "tmt.tencentcloudapi.com"

    # 实例化一个client选项，可选的，没有特殊需求可以跳过
    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    # 实例化要请求产品的client对象,clientProfile是可选的
    client = tmt_client.TmtClient(cred, "ap-beijing", clientProfile)

    words_list = read_column(source_file, source_column)

    # 实例化一个请求对象,每个接口都会对应一个request对象
    req = models.TextTranslateBatchRequest()
    params = {
        "Source": "auto",
        "Target": "zh",
        "ProjectId": 1286675,
        "SourceTextList": words_list
    }
    req.from_json_string(json.dumps(params))

    # 返回的resp是一个TextTranslateBatchResponse的实例，与请求对象对应
    resp = client.TextTranslateBatch(req)

    target_text_list = resp.TargetTextList  # 获取翻译结果值
    return target_text_list

In [17]:
# 翻译结果存储
def write_results(source_file, source_column, target_file, target_column):
    source_df = pd.read_excel(source_file) # 读取原文件为一个DataFrame
    target_text_list = translate_words(source_file, source_column) # 调用翻译函数，返回一个list
    target_df = pd.DataFrame(target_text_list, columns=[target_column]) # 将list转换为一个DataFrame，并指定列名
    new_df = source_df.join(target_df) # 将两个DataFrame按照索引合并为一个新的DataFrame
    new_df.to_excel(target_file, index=False) # 将新的DataFrame直接写入到文件中，指定index参数

In [22]:
inputFile = './input/translation.xlsx'
outputFile = './output/after_translation.xlsx'
# read_column(inputFile, "Method")
# translate_words(inputFile, "Keyword")
# df1 = pd.read_excel(inputFile)
# df1.columns

In [26]:
source_column = "Location" # 源文本所在列名，需要与源文件一致
target_column = "Location_Translation" # 翻译结果所在列名，会创建在目标文件中
write_results(inputFile, source_column, outputFile, target_column)

TencentCloudSDKException: [TencentCloudSDKException] code:InvalidParameter message:对于指定content-type=application/json协议下，请求内容不是合法的Json格式。 requestId:aafed8fe-e36d-42e3-9180-c9b6b3f1ad47

In [ ]:
source_column = "Method"
write_results(inputFile, outputFile, source_column)